In [2]:
import tkinter as tk
from tkinter import filedialog
import os
import re

def parse_srt(content):
    pattern = re.compile(r'(\d+)\n(.*?)\n(.*?)\n', re.DOTALL)
    blocks = []
    for match in pattern.finditer(content + "\n"):
        idx = int(match.group(1))
        time = match.group(2).strip()
        text = match.group(3).strip()
        blocks.append((idx, time, text))
    return blocks

def build_srt(blocks):
    return "\n".join(f"{idx}\n{time}\n{text}\n" for idx, time, text in blocks)

def shift_blocks(blocks, start_idx, end_idx, new_start_idx):
    """生成新的平移字幕 (向前移一格，最后补空)"""
    original = blocks[start_idx-1:end_idx]
    shifted = []
    for i in range(len(original)):
        idx = new_start_idx + i
        time = original[i][1]
        text = original[i+1][2] if i+1 < len(original) else ""
        shifted.append((idx, time, text))
    return shifted

def process_file(input_path, jp_end, cn_end):
    with open(input_path, "r", encoding="utf-8") as f:
        content = f.read()
    blocks = parse_srt(content)

    total = len(blocks)

    # 日语范围: 1 ~ jp_end
    new_jp = shift_blocks(blocks, 1, jp_end, total + 1)

    # 中文范围: (jp_end+1) ~ cn_end
    new_cn = shift_blocks(blocks, jp_end + 1, cn_end, total + len(new_jp) + 1)

    all_blocks = blocks + new_jp + new_cn
    output = build_srt(all_blocks)

    output_path = os.path.join(os.path.dirname(input_path), "output_shifted.srt")
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(output)

    print(f"✅ 已完成！输出文件保存为: {output_path}")
    print(f"📊 原始字幕总数: {total}")
    print(f"🈶 日语区间: 1 ~ {jp_end}，中文字幕区间: {jp_end+1} ~ {cn_end}")
    print(f"🆕 新增日语: {total+1} ~ {total+len(new_jp)}，新增中文: {total+len(new_jp)+1} ~ {total+len(new_jp)+len(new_cn)}")

def select_and_process_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="请选择SRT字幕文件",
        filetypes=[("SRT 字幕文件", "*.srt")]
    )
    if file_path:
        print(f"📂 你选择了文件: {file_path}")

        # 让用户输入区间
        jp_end = int(input("请输入日语字幕的结束编号: "))
        cn_end = int(input("请输入中文字幕的结束编号: "))

        process_file(file_path, jp_end, cn_end)
    else:
        print("❌ 没有选择任何文件。")

select_and_process_file()


📂 你选择了文件: E:/音乐制作/真夜中/剪辑视频/正片/出道前就活/ACAね弹唱-就活 - 副本.srt


请输入日语字幕的结束编号:  45
请输入中文字幕的结束编号:  90


✅ 已完成！输出文件保存为: E:/音乐制作/真夜中/剪辑视频/正片/出道前就活\output_shifted.srt
📊 原始字幕总数: 90
🈶 日语区间: 1 ~ 45，中文字幕区间: 46 ~ 90
🆕 新增日语: 91 ~ 135，新增中文: 136 ~ 180
